In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 5706.95021529,  5769.73404425],
        [ 5996.70758168,  5727.1912134 ],
        [ 7769.37515025,  6968.54352984],
        [ 6526.50903616,  7160.76378662],
        [ 8345.65321871,  7223.74468502],
        [ 6374.32733118,  5444.49174019],
        [ 6931.38800943,  4359.10654166],
        [ 3370.89827923,  4131.76453876],
        [ 4913.78397377,  3644.54925995],
        [ 2436.66325747,  5120.43726483],
        [ 9497.20235035,  4625.90787758],
        [ 6662.38765684,  8659.45857039],
        [ 2760.07845131,  6900.90798161],
        [ 3460.09542301,  2526.38019712],
        [ 5771.10891497,  6485.7370088 ],
        [ 2092.00514271,  4053.55624997],
        [ 6883.74001144,  4722.73799537],
        [ 3429.86933348,  4330.29629178],
        [ 4056.08296282,  6188.14457384],
        [ 4693.78104603,  5020.20677159],
        [ 3571.82719425,  3151.19618574],
        [ 3602.97974399,  3270.31054799],
        [ 4674.38277454,  5497.26710306],
        [ 4802.90575811,  6492.375

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e-02, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.73978595e+05  0.00000000e+00  4.23e+03 1.95e+02  9.67e+05     0s
   1   1.03323412e+06 -1.63017609e+06  8.84e+02 2.06e+02  2.43e+05     0s
   2   8.53296120e+05 -1.32499140e+06  8.84e-04 1.14e-13  3.89e+04     0s
   3   2.01675349e+04 -2.90998285e+05  8.85e-10 5.68e-14  5.5

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[4.3750165119324143e-10,
 6025.393759917874,
 4.802285497198828e-10,
 2.27246429599748e-06,
 1356.1732518291597,
 3.8099913036947164e-06,
 1.3962594929846191e-07,
 2334.1588611837246,
 316.7697148343589,
 4473.253281181975,
 2.2724642355521992e-06,
 269.3999441506581,
 3.8099954793124977e-06,
 3.831815156313504e-06,
 7359.678360605219,
 2129.656665243286,
 2.2724643335626357e-06,
 2.2724643213469874e-06,
 2064.8022963218405,
 7867.872287053946,
 3.8091811856492005e-06,
 2.9632967434925572e-05,
 4688.438999108004,
 2.3103284768194833e-06,
 4071.49256592244,
 32.80419466673305,
 30.785600361245884,
 4305.332146463127,
 2.3103232285374802e-06,
 2.3103237370992127e-06,
 2.3103284369622345e-06,
 1857.3204027230383,
 3629.2012149984603,
 5170.0605139528425,
 1838.7879658464294,
 9.16047337025518e-05,
 9.160452009488303e-05,
 3751.265976348696,
 682.8872155111709,
 2509.6683577414497,
 2738.0422854478898,
 824.7209265967696,
 4.427784957040045e-06,
 4.42778477410184e-06,
 8969.183877312813,
 